# Spherical harmonic expansion of planetary magnetic fields

## Motivation 

Simulating the motion of a charged planetary ring grain acting under planetary gravitational and magnetic fields. Depending on the planet in question, these magnetic fields can be pretty complicated and require more than a simple dipole expression. A standard way to express $\vec{B}$ is to use a spherical harmonic expansion of the potential $V$ and then express the magnetic field as its gradient: $\vec{B} = -\vec{\nabla}V$. The three components of the planetary magnetic field (where only contributions from internal sources, i.e. planetary dynamo and no external currents, are considered) are:

$$
\begin{align}
B_{r} &= \sum_{n=1}^{\infty} (n+1) \left( \frac{R_\text{p}}{r}\right)^{n+2} \sum_{m=0}^{n} [g_{n}^{m} \text{cos}(m\phi) + h_{n}^{m} \text{sin}(m\phi)]P^{m}_{n}(\text{cos}\theta), \\
B_{\theta} &= - \sum_{n=1}^{\infty} \left( \frac{R_\text{p}}{r}\right)^{n+2} \sum_{m=0}^{n} [g_{n}^{m} \text{cos}(m\phi) + h_{n}^{m} \text{sin}(m\phi)] \frac{dP^{m}_{n}(\text{cos}\theta)}{d\theta} , \tag{1} \\
B_{\phi} &= \frac{1}{\text{sin} \theta}\sum_{n=1}^{\infty} \left( \frac{R_\text{p}}{r}\right)^{n+2} \sum_{m=0}^{n} m[g_{n}^{m} \text{sin}(m\phi) - h_{n}^{m} \text{cos}(m\phi)]P^{m}_{n}(\text{cos}\theta).
\end{align}
$$

I follow the physics convention of $\theta$ being the polar angle measured from the rotation axis ($z$), $\phi$ being the azimuthal angle increasing in the direction of rotation (in $x-y$ plane), and $r$ being the radial distance: https://en.wikipedia.org/wiki/Spherical_coordinate_system#/media/File:3D_Spherical.svg .

$R_p$ is the radius of the planet, $n$ and $m$ are integers representing the order and degree of the expansion respectively. The $g_n^m, h_n^m$ are so-called Schmidt-normalised coefficients, which can be measured by spacecraft when performing fly-bys and typically are given in units of Tesla or Gauss. The $P_n^m$ are Schmidt-normalised Associated Legendre polynomials which I have had to write a separate Python module to calculate [assoc_legendre_schmidtnorm](./modules/assoc_legendre_schmidtnorm.py), but which is essentially the same as SymPy's built-in [`assoc_legendre.py`](https://docs.sympy.org/latest/modules/functions/special.html#sympy.functions.special.polynomials.assoc_legendre) but with a normalising factor. For reference, the Schmidt-normalised Associated Legendre polynomial generating expression looks like this:

$$
P^m_n(x) = \sqrt{(2-\delta^0_m)\frac{(n-m)!}{(n+m)!}}\mathcal{P}_n^m(x) \qquad for \quad |m| \leq n
$$

where $\mathcal{P}_n^m(x)$ are the (non-normalised) associated Legendre polynomials.
Here are some examples, from Connerney 1993 *Magnetic Fields of the Outer Planets* J. Geophys. Res.:

![img](./imgs/eg_schmidtnorm_leg.png)

I can calculate expressions for $P_n^m(cos\theta)$ and $\frac{dP_n^m}{d\theta}$ using my module `assoc_legendre_schmidtnorm.py` (which I adapted from SymPy's existing [`sympy.functions.special.polynomials.assoc_legendre()`](https://docs.sympy.org/latest/_modules/sympy/functions/special/polynomials.html) and substitute those together with the known $g_n^m$ and $h_n^m$ coeffiecients into the spherical harmonic expansion double summation expression above to find $B_r, B_\theta, B_\phi$.

Then, because my simulation works in Cartesian coords, I use a change-of-basis matrix to transform $(B_r, B_\theta, B_\phi)$ to $(B_x, B_y, B_z)$:

$$
\begin{bmatrix}
B_{x}\\ 
B_{y}\\ 
B_{z}
\end{bmatrix} =
\begin{bmatrix}
\text{sin}\theta\text{cos}\phi & \text{cos}\theta\text{cos}\phi  & -\text{sin}\phi\\ 
 \text{sin}\theta\text{sin}\phi & \text{cos}\theta\text{sin}\phi & \text{cos}\phi\\ 
 \text{cos}\theta & -\text{sin}\theta & 0
\end{bmatrix}
\begin{bmatrix}
B_{r}\\ 
B_{\theta}\\ 
B_{\phi}
\end{bmatrix}
$$

Doing this by hand is fine for simpler magnetic fields like Saturn's, which to first order is an aligned centred (axisymmetric) dipolar (n=1) magnetic field, or to higher order ($n=3$, octupole) is a vertically-offset axisymmetric field (known as the Z3 model). Note that many of the $g_n^m, h_n^m$ for that Z3 model are zero (only $g_1^0, g_2^0, g_3^0$ are non-zero) which simplifies things. In the table in the section below, I give expressions for those B fields.

However, for more complicated B fields like Uranus' (which involve more non-zero $g_n^m, h_n^m$ values to higher order) it's a lot more laborious to do by hand and open to error, so I wanted to find a way to automate it, especially as I don't want any bugs to creep in when it comes to typing up such long $B_x, B_y, B_z$ expressions into my integrator code. I thought if I could get SymPy to do it for me then I could simply print out the $B_x, B_y, B_z$ expressions and copy and paste those into my code (I have written Runge-Kutta and Bulirsch-Stoer methods which can take different force expressions, i.e. different $\vec{F}=q(\vec{v} \times \vec{B})$ to update the velocity step).

## Issues

The expressions output by my code for $B_x, B_y, B_z$ are quite long and complicated, and although OK for the purposes of copying and pasting into my existing velocity-update-step integrator code, it isn't ideal. It would be better to have SymPy simplify it, but I need guidance on whether to use expand, simplify, apart, powsimp, .... 

I would like to pull out factors of $\left(\frac{R_p}{r}\right)^3, \left(\frac{R_p}{r}\right)^4, \left(\frac{R_p}{r}\right)^5, \ldots$ using maybe something from this: https://docs.sympy.org/latest/modules/rewriting.html 

For reference, I include the pen-and-paper calculations and tests I ran for Saturn:


|  Order, degree of spherical harmonic expansion | Test Case | Actual pen-and-paper calculation  | Did it pass `pytest`?|
|-------------------|------------------------------|-------------------|-------------------|
| n = 1, m = 0 (aligned centred dipole) | sph. coords for any $\theta$ | $B_r = \frac{2g_0^1R_p^3cos(\theta)}{r^3}$, $B_\theta = \frac{g_0^1R_p^3sin(\theta)}{r^3}$, $B_\phi=0$ | YES |
| n = 1, m = 0 (aligned centred dipole) | Cart. coords for any $z$ | $\frac{3xzg_1^0R_p^3}{r^5}$, $\frac{3yzg_1^0R_p^3}{r^5}$, $\frac{g_1^0R_p^3}{r^5}(3z^2 - r^2)$ | YES|
| n = 1, m = 0 (aligned centred dipole) | sph. coords for $\theta = \frac{\pi}{2}$ | $B_r = 0$, $B_\theta = \frac{g_0^1R_p^3}{r^3}$, $B_\phi=0$ | YES |
| n = 1, m = 0 (aligned centred dipole) | Cart. coords for $z=0$ | $B_x = 0,  B_y = 0,  B_z = -\frac{g_1^0R_p^3}{r^3}$| YES | 
| n = 1,2,3, m = 0 (aligned vertically offset dipole Z3) | sph. coords for any $\theta$ |$B_r = 2g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \text{cos}\theta + \frac{9}{2}g_2^0\left(\frac{R_{Sat}}{r}\right)^4 (\text{cos}^2\theta - \frac{1}{3}) + 10g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \text{cos}\theta(\text{cos}^2\theta - \frac{9}{15}), \\
B_{\theta} = g_1^0 \left(\frac{R_{Sat}}{r}\right)^3\text{sin}\theta +  3g_2^0 \left(\frac{R_{Sat}}{r}\right)^4\text{cos}\theta\text{sin}\theta - \frac{3}{2}g_3^0 \left(\frac{R_{Sat}}{r}\right)^5\text{sin}\theta (1-5\text{cos}^2\theta), \\
B_{\phi} = 0$ | YES |
|n = 1,2,3, m = 0 (aligned vertically offset dipole Z3) | Cart. coords for any $z$ | $B_x  = 3g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \frac{xz}{r^2} + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15xz^2}{2r^3} - \frac{3x}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35xz^3}{2r^4} - \frac{15xz}{2r^2}\right) ,\\
B_y = 3g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \frac{yz}{r^2} + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15yz^2}{2r^3} - \frac{3y}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35yz^3}{2r^4} - \frac{15yz}{2r^2}\right) , \\
B_z =  g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \left(\frac{3z^2 -r^2}{r^2}\right) + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15z^3}{2r^3} - \frac{9z}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35z^4}{2r^4} - \frac{15z^2}{r^2} + \frac{3}{2} \right) $|  YES |
| n = 1,2,3, m = 0 (aligned vertically offset dipole Z3) | sph. coords for $\theta = \frac{\pi}{2}$ | $B_r = -\frac{3}{2}g_2^0\left(\frac{R_p}{r}\right)^4,\\
B_\theta = g_1^0\left(\frac{R_p}{r}\right)^3  - \frac{3}{2}g_3^0\left(\frac{R_p}{r}\right)^5 , \\
B_\phi = 0$  | YES |
| n = 1,2,3, m = 0 (aligned vertically offset dipole Z3) | Cart.coords for $z=0$ | $B_x = -\frac{3}{2}g_2^0\left(\frac{R_p}{r}\right)^4\frac{x}{r} , \\
B_y = -\frac{3}{2}g_2^0\left(\frac{R_p}{r}\right)^4\frac{y}{r} ,\\
B_z = -g_1^0\left(\frac{R_p}{r}\right)^3  + \frac{3}{2}g_3^0\left(\frac{R_p}{r}\right)^5$| YES |

## Current code

I have run correctness tests (using `pytest`) on my SymPy code that checks whether it produces the expected expressions for Saturn's dipole ($n=1$) and Z3 ($n=3$) models. It appears that it is working correctly as what I expect from pen-and-paper calculations and what SymPy finds are directly equivalent expressions:

![screenshot pytest](./imgs/screenshot_pytest.png)


Below, I provide code and tests that can be run using [`ipytest`](https://github.com/chmp/ipytest):

In [2]:
# Set up relevant stuff for testing purposes:
import pytest
import ipytest

# enable IPython magics for test execution
import ipytest.magics

# enable pytest's assertions
ipytest.config.rewrite_asserts = True

# set the filename
__file__ = 'sympy_Bfield.ipynb'


In [3]:
# Define the test cases by writing test_ functions that pytest reads in
# These test_ functions use my spherical harmonic code and change-of-basis code which I write
# into fixtures, calcB_sph() and B_sph_to_cart()

from sympy import symbols, Symbol, cos, sin, pi, sqrt, expand, cos, sin, acos, atan, atan2, ask, Q 
from sympy.simplify import simplify, trigsimp
from sympy.functions.elementary.complexes import Abs
# Import own module to calculate the Spherical Harmonic expansion for the B field components 
import sys
sys.path.insert(0, "./modules/")
import assoc_legendre_schmidtnorm # This is adapted from sympy.functions.special.polynomials' assoc_legendre (which is non-normalised)
                                  # Note that Sympy's built-in Associated Legendre function INCLUDES the Condon-Shortley phase of (-1)^m
                                  # But I've followed magnetism lit convention and NOT included the Condon-Shortley phase

# Define global variables
# Need to tell Sympy that theta is real, otherwise instead of sin(theta) it returns sqrt(sin^2(theta))
R_p, r, theta, phi, x, y, z = symbols('R_p r theta phi x y z', real=True, positive=True)

##################################   BEGIN FIXTURES   ##################################
# My tests work on some SymPy expressions so I generate them using fixtures:
@pytest.fixture()
# Everytime calcB_sph() function is injected into a test_ function, it will refer to the pass_arg_calcB_sph() function
# This is so that I can pass N (the max order of the spherical harmonic expansion) to the different test_ functions
def calcB_sph():
    def pass_arg_calcB_sph(N):
        """Function pass_arg_calcB_sph() does all the nitty gritty SymPy calculations to derive the expressions for B_r, B_theta, B_phi
        args:
            N (sympy symbolic int) : max value order of spherical harmonic expansion (ie the order that the summations will get summed up to)
        """

        assert ask(Q.integer(N) & Q.positive(N)), "N must be a positive integer"  # ensure that N is of sympy type <class 'int'> and positive
        theta, phi, r, x, y, z  = symbols('theta phi r x y z',real=True, positive=True) # Tell Sympy that it's real, otherwise instead of sin(theta) it returns sqrt(sin^2(theta))
        m, n = symbols('m n', integer=True, positive=True)
        g=Symbol('g', real=True, positive=True) # !potential bug: when I introduce g_n^m the realness & positivity is not enforced... but this isn't important
        h=Symbol('h', real=True, positive=True)

        # Initialise variables to store multipole expansion:
        B_r, B_theta, B_phi, R_p, r = symbols('B_r B_theta B_phi R_p r',real=True, positive=True) 
        # radial component of B-field, co-latitudinal component, and azimuthal component, planetary radius and radial distance of grain
     
        # Initialise components:   
        B_r=0
        B_theta=0
        B_phi=0

        # Create some intermediate symbolic variables to store sums over m and n, for each magnetic field component in turn:
        B_r_nth_term, B_r_nth_mth_term = symbols('B_r_nth_term B_r_nth_mth_term',real=True, positive=True)
        B_r_nth_term = 0
        B_r_nth_mth_term = 0

        B_theta_nth_term, B_theta_nth_mth_term = symbols('B_theta_nth_term B_theta_nth_mth_term',real=True, positive=True)
        B_theta_nth_term = 0
        B_theta_nth_mth_term = 0

        B_phi_nth_term, B_phi_nth_mth_term = symbols('B_phi_nth_term B_phi_nth_mth_term',real=True, positive=True)
        B_phi_nth_term = 0
        B_phi_nth_mth_term = 0

        # Use the summation formulae for the magnetic field components. For each n, sum from m=0 to m=n:
        for n in range(1,N+1): #Loop through from n=1 to max n specified
            # Create dummy variables for storing:
            B_r_nth_term = (n+1)*(R_p/r)**(n+2) 
            B_theta_nth_term = -(R_p/r)**(n+2) 
            B_phi_nth_term = 1/sin(theta)*(R_p/r)**(n+2) 

            # Loop through from m=0 to m=n, as n is held constant
            for m in range(0,n+1): 
                Pnm = assoc_legendre_schmidtnorm.assoc_legendre_schmidt_norm(n,m,cos(theta)) # Schmidt-normalised associated Legendre function
                # Next, stop sin^3(theta) being displayed as sin^2(theta)|sin(theta)|:
                Pnm = simplify(Pnm.expand(trig=True))#func=true
                Pnm = Pnm.xreplace({Abs(sin(theta)):sin(theta)}) # sort out the |sin(theta)| business!
                dPnm_dtheta = Pnm.diff(theta) # differentiate wrt theta
                # Sort out the Schmidt-coefficients, g_n^m, h_n^m:
                g_n__m = g # create a new sympy.core.symbol.Symbol variable to store g_n^m
                g_n__m = g_n__m.subs(g,'g_%i__%i'%(n,m)) #label the coeffient with the correct subscript and superscript
                h_n__m = h # create a new sympy.core.symbol.Symbol variable to store g_n^m
                h_n__m = h_n__m.subs(h,'h_%i__%i'%(n,m)) #label the coeffient with the correct subscript and superscript
            
                # Work out the nth,mth summation term-by-term each time the code loops and m increments by 1:
                B_r_nth_mth_term += B_r_nth_term*((g_n__m)*cos(m*phi) + (h_n__m)*sin(m*phi))*Pnm
                B_theta_nth_mth_term += B_theta_nth_term*((g_n__m)*cos(m*phi) + (h_n__m)*sin(m*phi))*dPnm_dtheta
                B_phi_nth_mth_term += B_phi_nth_term*m*((g_n__m)*sin(m*phi) - (h_n__m)*cos(m*phi))*Pnm
        
            # Store each incremented term in the B_subscript variable:
            B_r = B_r + B_r_nth_mth_term
            B_r_nth_mth_term = 0 # reset for the (n+1)th term that is about to be calculated
        
            B_theta = B_theta + B_theta_nth_mth_term
            B_theta_nth_mth_term = 0 
        
            B_phi = B_phi + B_phi_nth_mth_term
            B_phi_nth_mth_term = 0 
        
        return(B_r, B_theta, B_phi)

    return pass_arg_calcB_sph


# B_sph_to_cart transforms the B field components from spherical coordinates (r, theta, phi)
# to Cartesian coordinates (x,y,z) for use in my integrator's coord sys:
@pytest.fixture()
def B_sph_to_cart():
    def pass_arg_B_sph_to_cart(B_r, B_theta, B_phi):
        """ B_sph_to_cart:
        Transform the B field components from spherical coords (B_r, B_theta, B_phi)
        to Cartesian coords (B_x,, B_y, B_z) 
        Note the physics convention for spherical coords is used:
        - theta : polar angle, measured from the vertical z axis [0, pi]
        - phi : azimuthal angle, measured in the x-y plane [0, 2pi]
        - r : radial distance from planetary centre
        """
        
        theta, phi, r, x, y, z  = symbols('theta phi r x y z',real=True, positive=True) # Tell Sympy that it's real, otherwise instead of sin(theta) it returns sqrt(sin^2(theta))
        B_x = sin(theta)*cos(phi)*B_r + cos(theta)*cos(phi)*B_theta - sin(phi)*B_phi
        # NOTE: in the following , I comment OUT r = sqrt(x**2 + y**2 + z**2) because I want to keep things in terms of r
        B_x = B_x.subs({#r: sqrt(x**2 + y**2 + z**2),
                        #theta: acos(z/(sqrt(x**2 + y**2 + z**2))),
                        theta: acos(z/r),
                        phi: atan(y/x)})
        B_y = sin(theta)*sin(phi)*B_r + cos(theta)*sin(phi)*B_theta + cos(phi)*B_phi
        B_y = B_y.subs({#r: sqrt(x**2 + y**2 + z**2),
                        #theta: acos(z/(sqrt(x**2 + y**2 + z**2))),
                        theta: acos(z/r),
                        phi: atan(y/x)})
        B_z = cos(theta)*B_r - sin(theta)*B_theta + 0*B_phi
        B_z = B_z.subs({#r: sqrt(x**2 + y**2 + z**2),
                        #theta: acos(z/(sqrt(x**2 + y**2 + z**2))),
                        theta: acos(z/r),
                        phi: atan(y/x)})
        return (B_x, B_y, B_z)

    return pass_arg_B_sph_to_cart
##################################  END FIXTURES ##################################  


############################   BEGIN TEST_ FUNCTIONS   ############################  

#### Test n=1, m=0 axisymmetric aligned dipole field for equatorial plane:

## First in spherical coords... :
def test_aligned_centred_dipole_midplane_BrBthetaBphi(calcB_sph):
    print("Inside test_aligned_centred_dipole_midplane_BrBthetaBphi()")
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=1) #n=1 is dipole

    # Assign aligned dipole coefficients for Saturn, ie set certain g_n^m and all h_m^n to be zero 
    sympy_Br = sympy_Br.subs({'g_1__1':0, 
                'h_1__1':0,  'h_1__0':0,
                theta:pi /2   # midplane 
               })
    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 
                'h_1__1':0, 'h_1__0':0,  
                theta:pi/2  # midplane
               })

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 
                 'h_1__1':0, 'h _1__0':0,
                 theta:pi/2  # midplane
               })

    # Result from pen-and-paper:
    own_calc_Br = 0
    own_calc_Btheta = Symbol('g_1__0')*(R_p/r)**3
    own_calc_Bphi = 0

    test_eq_Br = own_calc_Br - sympy_Br
    assert(test_eq_Br==0)

    test_eq_Btheta = simplify(expand(own_calc_Btheta - sympy_Btheta))
    assert(test_eq_Btheta==0)
    
    test_eq = own_calc_Bphi - sympy_Bphi
    assert(test_eq==0) 

## ... next in Cartesian coords:
def test_aligned_centred_dipole_midplane_BxByBz(calcB_sph, B_sph_to_cart):
    print("Inside test_aligned_centred_dipole_midplane_BxByBz()") 
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=1) #n=1 is dipole

    # Assign aligned dipole coefficients for Saturn, ie setcertain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 
                'h_1__1':0,  'h_1__0':0,
                #theta:pi /2   # It's important to NOT sub in for theta before converting coord sys 
               })
    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 
                'h_1__1':0, 'h_1__0':0,  
               })

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 
                 'h_1__1':0, 'h _1__0':0,
               })

    # Convert coord sys:
    sympy_Bx, sympy_By, sympy_Bz = B_sph_to_cart(sympy_Br, sympy_Btheta, sympy_Bphi) # Important to sub in un-restricted theta B field components

    # Result from pen-and-paper:
    own_calc_Bx = 0
    own_calc_By = 0
    own_calc_Bz = -Symbol('g_1__0')*(R_p/r)**3

    # Midplane:
    sympy_Bx = sympy_Bx.subs(z, 0)
    sympy_By = sympy_By.subs(z, 0)
    sympy_Bz = sympy_Bz.subs(z, 0)

    test_eq_Bx = own_calc_Bx - sympy_Bx
    assert(test_eq_Bx==0)

    test_eq_By = own_calc_By - sympy_By
    assert(test_eq_By==0)
    
    test_eq_Bz = own_calc_Bz - sympy_Bz
    assert(test_eq_Bz==0) 


##### Test n=1, m=0 axisymmetric aligned dipole field for any z, theta (ie don't restrict to midplane):

## First in spherical coords... :
def test_aligned_centred_dipole_BrBthetaBphi(calcB_sph):
    print("Inside test_aligned_centred_dipole_BrBthetaBphi()")   
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=1) #n=1 is dipole

    # Assign aligned dipole coefficients for Saturn, ie set certain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 
                'h_1__1':0,  'h_1__0':0,
               })
    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 
                'h_1__1':0, 'h_1__0':0,  
               })

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 
                 'h_1__1':0, 'h _1__0':0,
               })

    # Result from pen-and-paper:
    own_calc_Br = 2*Symbol('g_1__0')*(R_p/r)**3*cos(theta)
    own_calc_Btheta = Symbol('g_1__0')*(R_p/r)**3*sin(theta)
    own_calc_Bphi = 0
 
    test_eq_Br = own_calc_Br - sympy_Br
    assert(test_eq_Br==0)

    test_eq_Btheta = own_calc_Btheta - sympy_Btheta
    assert(test_eq_Btheta==0)
    
    test_eq = own_calc_Bphi - sympy_Bphi
    assert(test_eq==0) 

## ... next in Cartesian coords:
def test_aligned_centred_dipole_BxByBz(calcB_sph, B_sph_to_cart):
    print("Inside  test_aligned_centred_dipole_BxByBz()") 
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=1) #n=1 is dipole

    # Assign aligned dipole coefficients for Saturn, ie setcertain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 
                'h_1__1':0,  'h_1__0':0,
               })
    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 
                'h_1__1':0, 'h_1__0':0,  
               })

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 
                 'h_1__1':0, 'h _1__0':0,
               })

    # Convert coord sys:
    sympy_Bx, sympy_By, sympy_Bz = B_sph_to_cart(sympy_Br, sympy_Btheta, sympy_Bphi) # Important to sub in un-restricted theta B field components
  
    # Result from pen-and-paper:
    own_calc_Bx = 3*Symbol('g_1__0')*(x*z/r**2)*(R_p/r)**3
    own_calc_By = 3*Symbol('g_1__0')*(y*z/r**2)*(R_p/r)**3
    own_calc_Bz = Symbol('g_1__0')*(R_p/r)**3*(3*z**2 - r**2)/r**2

    # Sub in for r:
    sympy_Bx = simplify(sympy_Bx.subs(r, sqrt(x**2+y**2+z**2)))
    sympy_By = simplify(sympy_By.subs(r, sqrt(x**2+y**2+z**2)))
    
    own_calc_Bx = own_calc_Bx.subs(r, sqrt(x**2 + y**2 + z**2))
    own_calc_By = own_calc_By.subs(r, sqrt(x**2 + y**2 + z**2))

    test_eq_Bx = own_calc_Bx - sympy_Bx
    assert(test_eq_Bx==0)

    test_eq_By = own_calc_By - sympy_By
    assert(test_eq_By==0)
    
    test_eq_Bz = simplify(own_calc_Bz - sympy_Bz)
    assert(test_eq_Bz==0) 


#### Test n=1,2,3, m=0 axisymmetric vertically offset dipole field for equatorial plane (z=0):

## First in spherical coords... :
def test_offset_dipole_midplane_BrBthetaBphi(calcB_sph):
    print("Inside test_offset_dipole_midplane_BrBthetaBphi()")
    # Call my sympy code to obtain expressions for field components:
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=3) #n=3 is octupole

    # Assign aligned dipole coefficients for Saturn Z3, ie set certain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
                theta:pi/2 # midplane
               }) 

    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
                theta:pi/2 # midplane
               }) 

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
                theta:pi/2 # midplane
               }) 


    # Result from pen-and-paper:
    own_calc_Br = -3/2*Symbol('g_2__0')*(R_p/r)**4
    own_calc_Btheta = Symbol('g_1__0')*(R_p/r)**3 - 3/2*Symbol('g_3__0')*(R_p/r)**5
    own_calc_Bphi = 0 

    test_eq_Br = own_calc_Br - sympy_Br
    assert(test_eq_Br==0)

    test_eq_Btheta = own_calc_Btheta - sympy_Btheta
    assert(test_eq_Btheta==0)
    
    test_eq = own_calc_Bphi - sympy_Bphi
    assert(test_eq==0) 

## ... next in Cartesian coords:
def test_offset_dipole_midplane_BxByBz(calcB_sph, B_sph_to_cart):
    print("Inside test_offset_dipole_midplane_BxByBz()") 
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=3) #n=3 is octupole
    
    # Assign aligned dipole coefficients for Saturn Z3, ie set certain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    # Convert coord sys:
    sympy_Bx, sympy_By, sympy_Bz = B_sph_to_cart(sympy_Br, sympy_Btheta, sympy_Bphi) # Important to sub in un-restricted theta B field components

    # Result from pen-and-paper:
    own_calc_Bx = -3/2*Symbol('g_2__0')*(R_p/r)**4*x/r 
    own_calc_By = -3/2*Symbol('g_2__0')*(R_p/r)**4*y/r
    own_calc_Bz = -Symbol('g_1__0')*(R_p/r)**3 + 3/2*Symbol('g_3__0')*(R_p/r)**5

    # Midplane:
    sympy_Bx = sympy_Bx.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})
    sympy_By = sympy_By.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})
    sympy_Bz = sympy_Bz.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})
    own_calc_Bx = own_calc_Bx.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})
    own_calc_By = own_calc_By.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})
    own_calc_Bz = own_calc_Bz.subs({z:0, r:sqrt(x**2 + y**2 + z**2)})

    test_eq_Bx = simplify(own_calc_Bx - sympy_Bx)
    assert(test_eq_Bx==0)

    test_eq_By = simplify(own_calc_By - sympy_By)
    assert(test_eq_By==0)
    
    test_eq_Bz = own_calc_Bz - sympy_Bz
    assert(test_eq_Bz==0) 

    
#### Test n=1,2,3, m=0 axisymmetric vertically offset dipole field for any z, theta:

## First in spherical coords...:
def test_offset_dipole_BrBthetaBphi(calcB_sph):
    print("Inside test_offset_dipole_BrBthetaBphi()")   
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=3) #n=3 is octupole

    # Assign aligned dipole coefficients for Saturn Z3, ie set certain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    # Result from pen-and-paper:
    own_calc_Br = 2*Symbol('g_1__0')*(R_p**3/r**3)*cos(theta) + (9/2)*Symbol('g_2__0')*(R_p**4/r**4)*( cos(theta)**2 - (1/3)) + 10*Symbol('g_3__0')*(R_p**5/r**5)*cos(theta)*((cos(theta))**2 - (9/15))  
    own_calc_Btheta = Symbol('g_1__0')*(R_p**3/r**3)*sin(theta) + 3*Symbol('g_2__0')*(R_p**4/r**4)*cos(theta)*sin(theta) - (3/2)*Symbol('g_3__0')*(R_p**5/r**5)*sin(theta)*(1 - 5*(cos(theta))**2)
    own_calc_Bphi = 0
 
    test_eq_Br = trigsimp(own_calc_Br - sympy_Br)
    assert(test_eq_Br==0)

    test_eq_Btheta = trigsimp(own_calc_Btheta - sympy_Btheta)
    assert(test_eq_Btheta==0)
    
    test_eq = own_calc_Bphi - sympy_Bphi
    assert(test_eq==0) 


# ... next in Cartesian coords:
def test_offset_dipole_BxByBz(calcB_sph, B_sph_to_cart):
    print("Inside test_offset_dipole_BxByBz()") 
    # Call my sympy code to obtain expressions for field components: 
    sympy_Br, sympy_Btheta, sympy_Bphi = calcB_sph(N=3) #n=3 is dipole

    # Assign aligned dipole coefficients for Saturn Z3, ie set certain g_n^m and all h_m^n to be zero (ie eliminate those terms)
    sympy_Br = sympy_Br.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Btheta = sympy_Btheta.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    sympy_Bphi = sympy_Bphi.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) 

    # Convert coord sys:
    sympy_Bx, sympy_By, sympy_Bz = B_sph_to_cart(sympy_Br, sympy_Btheta, sympy_Bphi) 
 
    # Result from pen-and-paper:
    own_calc_Bx = 3*Symbol('g_1__0')*(R_p**3/r**3)*x*z/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (15*x*z**2)/(2*r**3) - 3*x/(2*r)) + Symbol('g_3__0')*(R_p**5/r**5)*( (35*x*z**3) / (2*r**4) - (15*x*z)/(2*r**2) ) 
    own_calc_By = 3*Symbol('g_1__0')*(R_p**3/r**3)*y*z/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (15*y*z**2) / (2*r**3) - 3*y/(2*r)) + Symbol('g_3__0')*(R_p**5/r**5)*( (35*y*z**3) / (2*r**4) - (15*y*z)/(2*r**2) )
    #own_calc_Bz = Symbol('g_1__0')*(R_p**3/r**3)*(3*z**2-r**2)/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (9*z**3)/(2*r**3) - (3*z)/(2*r) - (3*z/r)*((x**2+y**2)/(r**2)) ) + Symbol('g_3__0')*(R_p**5/r**5)*( (10*z**4) / (r**4) - (6*z**2) / (r**2) + 3/2*(x**2+y**2)/r**2 - 15/2*(z**2/r**2)*(x**2+y**2)/r**2)
    own_calc_Bz = Symbol('g_1__0')*(R_p**3/r**3)*(3*z**2-r**2)/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (9*z**3)/(2*r**3) - (3*z)/(2*r) - (3*z/r)*((x**2+y**2)/(r**2)) ) + Symbol('g_3__0')*(R_p**5/r**5)*( (10*z**4) / (r**4) - (6*z**2) / (r**2) + 3/2*(x**2+y**2)/r**2 - 15/2*(z**2/r**2)*(x**2+y**2)/r**2)

    # Sub in for r:
    sympy_Bx = sympy_Bx.subs(r, sqrt(x**2+y**2+z**2))
    sympy_By = sympy_By.subs(r, sqrt(x**2+y**2+z**2))
    sympy_Bz = sympy_Bz.subs(r, sqrt(x**2+y**2+z**2))

    own_calc_Bx = own_calc_Bx.subs(r, sqrt(x**2 + y**2 + z**2))
    own_calc_By = own_calc_By.subs(r, sqrt(x**2 + y**2 + z**2))
    own_calc_Bz = own_calc_Bz.subs(r, sqrt(x**2 + y**2 + z**2))


    test_eq_Bx = simplify(expand(own_calc_Bx - sympy_Bx))
    assert(test_eq_Bx==0)

    test_eq_By = simplify(expand(own_calc_By - sympy_By))
    assert(test_eq_By==0)
    
    test_eq_Bz = simplify(expand(own_calc_Bz - sympy_Bz))
    assert(test_eq_Bz==0) 
    
print("Reached end of cell")

Have imported assoc_legendre_schmidtnorm.py
Reached end of cell


In [4]:
# execute the tests via pytest, arguments are passed to pytest
ipytest.run('-qq')
print("Done!")

........                                                                                                                               [100%]
Done!


I've tried looking through https://docs.sympy.org/latest/tutorial/simplification.html, but still having trouble coercing it into a nicer format. The format I would like is having the $\left(\frac{R_p}{r}\right)^a$ factors out the front of the expressions, e.g. something that looks like my pen-and-paper calculations, for Z3 model:

$$
B_x  = 3g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \frac{xz}{r^2} + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15xz^2}{2r^3} - \frac{3x}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35xz^3}{2r^4} - \frac{15xz}{2r^2}\right) ,\\
B_y = 3g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \frac{yz}{r^2} + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15yz^2}{2r^3} - \frac{3y}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35yz^3}{2r^4} - \frac{15yz}{2r^2}\right) , \\
B_z =  g_1^0\left(\frac{R_{Sat}}{r}\right)^3 \left(\frac{3z^2 -r^2}{r^2}\right) + g_2^0\left(\frac{R_{Sat}}{r}\right)^4 \left(\frac{15z^3}{2r^3} - \frac{9z}{2r} \right) + g_3^0\left(\frac{R_{Sat}}{r}\right)^5 \left(\frac{35z^4}{2r^4} - \frac{15z^2}{r^2} + \frac{3}{2} \right)
$$

For instance, below I show my SymPy Z3 results, where none of them look like my pen-and-paper calculations (although mathematically equivalent as shown by my if-statements and the tests above):

In [5]:
# Z3 (spherical harmonic expansion with non-zero g_1^0, g_2^0, g_3^0 )
# Any z (don't substitute a particular value for theta)

from sympy import init_printing, latex, pi, sqrt, expand , powsimp, collect
from IPython.display import display, Math , Latex

from sympy.functions.special.polynomials import assoc_legendre
from sympy import cos, sin, acos, atan, atan2, Symbol, symbols 
from sympy.functions.elementary.complexes import Abs
from sympy.simplify import simplify, trigsimp
from sympy.core import sympify

import sys
sys.path.insert(0, "./modules/")

import assoc_legendre_schmidtnorm 
# Note that Sympy's built-in Associated Legendre function INCLUDES the Condon-Shortley phase of (-1)^m 
# But I've followed magnetism literature convention to NOT include the Condon-Shortley phase in the Schmidt-normalised polynomials

init_printing() # initialise pretty printing

theta, phi, r, x, y, z  = symbols('theta phi r x y z',real=True, positive=True) # Tell Sympy that it's real, otherwise instead of sin(theta) it returns sqrt(sin^2(theta))
m, n, N = symbols('m n N', integer=True, positive=True)
g=Symbol('g', real=True, positive=True)
h=Symbol('h', real=True, positive=True)

N = 3 # octupole

# Initialise variables to store multipole expansion:
B_r, B_theta, B_phi, R_p, r = symbols('B_r B_theta B_phi R_p r',real=True, positive=True) # radial component of B-field, co-latitudinal component, and azimuthal component, planetary radius and radial distance of grain
B_r=0
B_theta=0
B_phi=0

# Create some intermediate symbolic variables to store sums over m and n, for each magnetic field component in turn:
B_r_nth_term, B_r_nth_mth_term = symbols('B_r_nth_term B_r_nth_mth_term',real=True, positive=True)
B_r_nth_term = 0
B_r_nth_mth_term = 0

B_theta_nth_term, B_theta_nth_mth_term = symbols('B_theta_nth_term B_theta_nth_mth_term',real=True, positive=True)
B_theta_nth_term = 0
B_theta_nth_mth_term = 0

B_phi_nth_term, B_phi_nth_mth_term = symbols('B_phi_nth_term B_phi_nth_mth_term',real=True, positive=True)
B_phi_nth_term = 0
B_phi_nth_mth_term = 0


# Use the summation formulae for the magnetic field components. For each n, sum from m=0 to m=n:
for n in range(1,N+1): #Loop through from n=1 to n=N
    #print("n = ", n)
    B_r_nth_term = (n+1)*(R_p/r)**(n+2) # create dummy variable for storing
    B_theta_nth_term = -(R_p/r)**(n+2) # create dummy variable for storing
    B_phi_nth_term = 1/sin(theta)*(R_p/r)**(n+2) # create dummy variable for storing

    for m in range(0,n+1): # Loop through from m=0 to m=n, as n is held constant
        Pnm = assoc_legendre_schmidtnorm.assoc_legendre_schmidt_norm(n,m,cos(theta)) # Schmidt-normalised associated Legendre function
        # Next, stop sin^3(theta) being displayed as sin^2(theta)|sin(theta)|
        Pnm = simplify(Pnm.expand(trig=True))#func=true
        Pnm = Pnm.xreplace({Abs(sin(theta)):sin(theta)}) # sort out the |sin(theta)| business!
        dPnm_dtheta = Pnm.diff(theta) # differentiate wrt theta
        # Sort out the Schmidt-coefficients, g_n^m, h_n^m:
        g_n__m = g # create a new sympy.core.symbol.Symbol variable to store g_n^m
        g_n__m = g_n__m.subs(g,'g_%i__%i'%(n,m)) #label the coeffient with the correct subscript and superscript
        h_n__m = h # create a new sympy.core.symbol.Symbol variable to store g_n^m
        h_n__m = h_n__m.subs(h,'h_%i__%i'%(n,m)) #label the coeffient with the correct subscript and superscript
        
        # Work out the nth,mth summation term-by-term each time the code loops and m increments by 1:
        B_r_nth_mth_term += B_r_nth_term*((g_n__m)*cos(m*phi) + (h_n__m)*sin(m*phi))*Pnm
        B_theta_nth_mth_term += B_theta_nth_term*((g_n__m)*cos(m*phi) + (h_n__m)*sin(m*phi))*dPnm_dtheta
        B_phi_nth_mth_term += B_phi_nth_term*m*((g_n__m)*sin(m*phi) - (h_n__m)*cos(m*phi))*Pnm

    # Store each incremented term in the B_subscript variable:
    B_r = B_r + B_r_nth_mth_term
    B_r_nth_mth_term = 0 # reset for the (n+1)th term
    
    B_theta = B_theta + B_theta_nth_mth_term
    B_theta_nth_mth_term = 0
    
    B_phi = B_phi + B_phi_nth_mth_term
    B_phi_nth_mth_term = 0 
    

print("********************* B_r *********************")
# Set Saturn's g_1^1, g_2^1,..., h_n^m... etc coefficients to zero for Z3 model
B_r = B_r.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
                #theta:pi/2 # It's important to NOT sub in for theta BEFORE converting coord system
               })

print("check Sympy final expression: ")
display(Math(r"B_r = ") , B_r)

# Ensure that can convert to LaTeX syntax:
##B_r_latex = latex(B_r)
##print("Latex output:", B_r_latex)

# Ensure can use standard Python syntax
#print("Standard print output:\n ", B_r)

print("\n********************* B_theta *********************")
B_theta = B_theta.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) # Eliminate certain g_n^m and all h_m^n 

print("check Sympy final expression: ")

display(Math(r"B_\theta = ") , B_theta)

# Ensure that can convert to LaTeX syntax:
##B_theta_latex = latex(B_theta)
##print("Latex output:", B_theta_latex)

# Ensure can use standard Python syntax
#print("Standard print output:\n ", B_theta)

print("\n********************* B_phi *********************")
B_phi = B_phi.subs({'g_1__1':0, 'g_2__1':0, 'g_2__2':0, 'g_3__1':0,'g_3__2':0,'g_3__3':0, 
                'h_1__1':0, 'h_2__1':0, 'h_2__2':0, 'h_3__1':0,'h_3__2':0,'h_3__3':0,
                'h_1__0':0, 'h_2__0':0, 'h_3__0':0,
               }) # Eliminate certain g_n^m and all h_m^n 

print("check Sympy final expression: ")
display(Math(r"B_\phi = ") , B_phi)

# Ensure that can convert to LaTeX syntax:
##B_phi_latex = latex(B_phi)
##print("Latex output:", B_phi_latex)

# Ensure can use standard Python syntax
##print("Standard print output:\n ", B_phi)


################# Find B_x, B_y,B_z from B_r, B_theta, B_phi using change-of-basis matrix component-wise ################## 

# Define r in terms of x,y,z
#r=sqrt(x**2 + y**2 + z**2)

B_x = sin(theta)*cos(phi)*B_r + cos(theta)*cos(phi)*B_theta - sin(phi)*B_phi
B_x=B_x.subs({#r: sqrt(x**2 + y**2 + z**2),
              #theta: acos(z/(sqrt(x**2 + y**2 + z**2))),
              theta: acos(z/r),
              phi: atan(y/x)})

#B_x = B_x.subs(r, sqrt(x**2 + y**2 + z**2))
display("Unsimplified ", Math(r"B_x = "), B_x)

B_y = sin(theta)*sin(phi)*B_r + cos(theta)*sin(phi)*B_theta + cos(phi)*B_phi
B_y = B_y.subs({theta: acos(z/r),
                phi: atan(y/x)})

display("Unsimplified ", Math(r"B_y = "), B_y)

B_z = cos(theta)*B_r - sin(theta)*B_theta + 0*B_phi
B_z = B_z.subs({theta: acos(z/r), 
                phi: atan(y/x)})

display("Unsimplified ", Math(r"B_z = "), B_z)


############################## CHECK RESULTS ##############################

print("Checking n=3, Z3 B field:")

own_calc_B_x = 3*Symbol('g_1__0')*(R_p**3/r**3)*x*z/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (15*x*z**2)/(2*r**3) - 3*x/(2*r)) + Symbol('g_3__0')*(R_p**5/r**5)*( (35*x*z**3) / (2*r**4) - (15*x*z)/(2*r**2) ) 

print("Pen & paper calculation for B_x:")
display(own_calc_B_x)

# Define r in terms of x,y,z so that SymPy understands that r^2 is equivalent to x^2 + y^2 + z^2
if simplify(expand(own_calc_B_x.subs(r,sqrt(x**2 + y**2 + z**2)) - B_x.subs(r, sqrt(x**2 + y**2 + z**2)) )) == 0 :
    print("Passed test: B_x")
else:
    print("Failed test: B_x")
   
    
own_calc_B_y = 3*Symbol('g_1__0')*(R_p**3/r**3)*y*z/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (15*y*z**2) / (2*r**3) - 3*y/(2*r)) + Symbol('g_3__0')*(R_p**5/r**5)*( (35*y*z**3) / (2*r**4) - (15*y*z)/(2*r**2) )

print("Pen & paper calculation for B_y:")
display(own_calc_B_y)

if simplify(expand(own_calc_B_y.subs(r,sqrt(x**2 + y**2 + z**2)) - B_y.subs(r, sqrt(x**2 + y**2 + z**2)) )) == 0 :
    print("Passed test: B_y")
else:
    print("Failed test: B_y")

    
own_calc_B_z = Symbol('g_1__0')*(R_p**3/r**3)*(3*z**2-r**2)/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (15*z**3)/(2*r**3) - (9*z)/(2*r) ) + Symbol('g_3__0')*(R_p**5/r**5)*( (35*z**4) / (2*r**4) - (15*z**2) / (r**2) + 3/2 )
#equivalent expression:
#own_calc_B_z = Symbol('g_1__0')*(R_p**3/r**3)*(3*z**2-r**2)/r**2 + Symbol('g_2__0')*(R_p**4/r**4)*( (9*z**3)/(2*r**3) - (3*z)/(2*r) - (3*z/r)*((x**2+y**2)/(r**2)) ) + Symbol('g_3__0')*(R_p**5/r**5)*( (10*z**4) / (r**4) - (6*z**2) / (r**2) + 3/2*(x**2+y**2)/r**2 - 15/2*(z**2/r**2)*(x**2+y**2)/r**2)

print("Pen & paper calculation for B_z:")
display(own_calc_B_z)

if simplify(expand(own_calc_B_z.subs(r,sqrt(x**2 + y**2 + z**2)) - B_z.subs(r, sqrt(x**2 + y**2 + z**2)) )) == 0 :
    print("Passed test: B_z")
else:
    print("Failed test: B_z")

############################## END CHECK RESULTS ##############################

######################### TRY VARIOUS SIMPLIFICATIONS #########################
# Next, try coercing expressions into prettier format:
print("\n Now try simplifying the Bx, By, Bz sympy expressions that will be input into the integrator functions: \n ")
print("\n", "*"*30, "sympy B_x" ,"*"*30,"\n")
print("simplify:")
display(simplify(B_x))
print("powsimp:")
display(powsimp(B_x, deep=True))
print("trigsimp:")
display(trigsimp(B_x))
print("\n")

print("\n", "*"*30, "sympy B_y" ,"*"*30,"\n")
print("simplify:")
display(simplify(B_y))
print("powsimp:")
display(powsimp(B_y, deep=True))
print("trigsimp:")
display(trigsimp(B_y))
print("\n")

print("\n", "*"*30, "sympy B_z" ,"*"*30,"\n")
print("simplify:")
display(simplify(B_z))
print("powsimp:")
display(powsimp(B_z, deep=True))
print("trigsimp:")
display(trigsimp(B_z))
print("\n")

print("Done!")

********************* B_r *********************
check Sympy final expression: 


<IPython.core.display.Math object>

                                             ⎛     2       ⎞                  
                                       4     ⎜3⋅cos (θ)   1⎟                  
    5     ⎛     2       ⎞          3⋅Rₚ ⋅g⁰₂⋅⎜───────── - ─⎟       3          
2⋅Rₚ ⋅g⁰₃⋅⎝5⋅cos (θ) - 3⎠⋅cos(θ)             ⎝    2       2⎠   2⋅Rₚ ⋅g⁰₁⋅cos(θ
──────────────────────────────── + ───────────────────────── + ───────────────
                5                               4                      3      
               r                               r                      r       

 
 
 
)
─
 
 


********************* B_theta *********************
check Sympy final expression: 


<IPython.core.display.Math object>

...

powsimp:


               ⎛        ⎛                                       ________⎞     
               ⎜        ⎜                                      ╱      2 ⎟     
               ⎜        ⎜                               2     ╱      z  ⎟     
               ⎜        ⎜      ________              5⋅z ⋅   ╱   1 - ── ⎟     
               ⎜        ⎜     ╱      2  ⎛       2⎞          ╱         2 ⎟     
               ⎜  5     ⎜    ╱      z   ⎜3   5⋅z ⎟        ╲╱         r  ⎟     
      ________ ⎜Rₚ ⋅g⁰₃⋅⎜   ╱   1 - ── ⋅⎜─ - ────⎟ - ───────────────────⎟   3⋅
     ╱      2  ⎜        ⎜  ╱         2  ⎜2      2⎟             2        ⎟     
    ╱      z   ⎜        ⎝╲╱         r   ⎝    2⋅r ⎠            r         ⎠     
   ╱   1 - ── ⋅⎜───────────────────────────────────────────────────────── - ──
  ╱         2  ⎜                             5                                
╲╱         r   ⎝                            r                                 

                                                 ⎞ 

trigsimp:


                 ⎛          ⎛                                           ______
                 ⎜          ⎜                                          ╱      
                 ⎜          ⎜                                   2     ╱      z
                 ⎜          ⎜                     ________   5⋅z ⋅   ╱   1 - ─
                 ⎜          ⎜  ⎛         2⎞      ╱      2           ╱         
                 ⎜    5     ⎜  ⎜  3   5⋅z ⎟     ╱      z          ╲╱         r
        ________ ⎜  Rₚ ⋅g⁰₃⋅⎜- ⎜- ─ + ────⎟⋅   ╱   1 - ──  - ─────────────────
       ╱      2  ⎜          ⎜  ⎜  2      2⎟   ╱         2              2      
      ╱      z   ⎜          ⎝  ⎝      2⋅r ⎠ ╲╱         r              r       
-    ╱   1 - ── ⋅⎜- ──────────────────────────────────────────────────────────
    ╱         2  ⎜                                 5                          
  ╲╱         r   ⎝                                r                           

__⎞                                                



Done!


Given that more complicated planetary magnetic field expressions will look even more convoluted, I would like SymPy to factor out $\left(\frac{R_p}{r}\right)^a$ terms ...